# Grid simulator for Simulation Fitting Algorithms

In [2]:
import os
from GPU_Classes import *
from Image_Manager import *
from Polarization_Obtention_Algorithms import Gradient_Algorithm
import numpy as np
import json
import cv2
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Set the PARAMETERS ############################################
##################################################################
experiment_name="540_Basler_like_phiCR_0"
output_directory=f"./OUTPUT/LIBRARIES_OF_THEORETICAL_CR/{experiment_name}/"
os.chdir(f"/home/oiangu/Desktop/Conical_Refraction_Polarimeter/")
os.makedirs(f"{output_directory}/{experiment_name}/", exist_ok=True)

randomization_seed=666
image_depth=8 # or 16 bit per pixel
image_shortest_side=540
saturation=1
significant_decimal=3

# 1. SIMULATION ####################################################
# Ring parameters to test (each will be a different simulation)
phiCR_s=np.linspace(0,360,360*10**significant_decimal+1)*np.pi/180
R0_s=[167] #np.linspace(70,180,40) # in pxels
#rho_0s=np.array([2,3,4,5,6,7,9,11,14,20])
w0_s=[30] #np.linspace(8,50,40)
rho_0s=R0_s/w0_s


resolution_side_nx=image_shortest_side # generated images will be resolution_side x resolution_side
# Other parameters
max_k=50
num_k=1200
sim_chunk_ax=image_shortest_side


# 4. GRAVICENTER iX and PROFILES ######################################
X=int(resolution_side_nx*1.4/2)


##################################################
##################################################

# General considerations
im_type=np.uint16 if image_depth==16 else np.uint8
max_intensity=65535 if image_depth==16 else 255
np.random.seed(randomization_seed)

In [ ]:
# GENERAL ROUTINES #################################
def compute_intensity_gravity_center(image):
    """
        Expects input image to be an array of dimensions [h, w].
        It will return an array of gravity centers [2(h,w)] in pixel coordinates
        Remember that pixel coordinates are set equal to numpy indices

    """
    # image wise total intensity and marginalized inensities for weighted sum
    intensity_in_w = np.sum(image, axis=0) # weights for x [raw_width]
    intensity_in_h = np.sum(image, axis=1) # weights for y [raw_height]
    total_intensity = intensity_in_h.sum()

    # Compute mass center for intensity
    # [2] (h_center,w_center)
    return np.nan_to_num( np.stack(
        (np.dot(intensity_in_h, np.arange(image.shape[0]))/total_intensity,
         np.dot(intensity_in_w, np.arange(image.shape[1]))/total_intensity)
        ) )

def compute_raw_to_centered_iX(image, X):

    g_raw = compute_intensity_gravity_center(image)
    # crop the iamges with size (X+1+X)^2 leaving the gravity center in
    # the central pixel of the image. In case the image is not big enough for the cropping,
    # a 0 padding will be made.
    centered_image = np.zeros( (2*X+1, 2*X+1),  dtype = image.dtype )

    # we round the gravity centers to the nearest pixel indices
    g_index_raw = np.rint(g_raw).astype(int) #[N_images, 2]

    # obtain the slicing indices around the center of gravity
    # TODO -> make all this with a single array operation by stacking the lower and upper in
    # a new axis!!
    # [ 2 (h,w)]
    unclipped_lower = g_index_raw[:]-X
    unclipped_upper = g_index_raw[:]+X+1
    # unclippde could get out of bounds for the indices, so we clip them
    lower_bound = np.clip( unclipped_lower, a_min=0, a_max=image.shape)
    upper_bound = np.clip( unclipped_upper, a_min=0, a_max=image.shape)
    # we use the difference between the clipped and unclipped to get the necessary padding
    # such that the center of gravity is left still in the center of the image
    padding_lower = lower_bound-unclipped_lower
    padding_upper = upper_bound-unclipped_upper

    # crop the image
    centered_image[padding_lower[0]:padding_upper[0] or None,
                                    padding_lower[1]:padding_upper[1] or None ] = \
                  image[lower_bound[0]:upper_bound[0],
                                      lower_bound[1]:upper_bound[1]]
    return centered_image
    '''
    else:
        # We compute the center of gravity of the cropped images, if everything was made allright
        # they should get just centered in the central pixels number X+1 (index X)
        g_centered = compute_intensity_gravity_center(centered_image)

        # We now compute a floating translation of the image so that the gravicenter is exactly
        # centered at pixel (607.5, 607.5) (exact center of the image in pixel coordinates staring
        # form (0,0) and having size (607*2+1)x2), instead of being centered at the beginning of
        # around pixel (607,607) as is now
        translate_vectors = X+0.5-g_centered #[ 2(h,w)]
        T = np.float64([[1,0, translate_vectors[1]], [0,1, translate_vectors[0]]])
        return cv2.warpAffine( centered_image, T, (X*2+1, X*2+1),
                    flags=interpolation_flag) # interpolation method
    '''

In [ ]:
# Initialize the vigilant
try:
    phase_vigilant = json.load(open(vig_path))
except:
    phase_vigilant = {'R0s':[], 'w0s':[], 'phiCRs':[], 'grav':[], 'rel_path':[]}

# Set the objects ready ##################
# The image manager
image_loader = Image_Manager(mode=X, interpolation_flag=None)
# Define the Gradient algorithm
gradient_algorithm = Gradient_Algorithm(image_loader,
        rad_min_Grav, rad_max_Grav,
        initial_guess_delta_pix,
        use_exact_gravicenter)
image_container=np.zeros( (1, 2*X+1, 2*X+1), dtype=np.float64)
image_names=['a']
# The simulator object
simulator=RingSimulator_Optimizer_GPU( n=1.5, a0=1.0, max_k=max_k, num_k=num_k, nx=resolution_side_nx, sim_chunk_x=sim_chunk_ax, sim_chunk_y=sim_chunk_ax)

cols = np.broadcast_to( np.arange(X*2+1), (X*2+1,X*2+1)) #[h,w]
rows = cols.swapaxes(0,1) #[h,w]

# Execute the stuff #####################
i=1
for phi_CR in phiCR_s:
    for R0 in R0_s:
        for w0 in w0_s:
            ID=f"phiCR_{phi_CR}_R0_{R0}_w0_{w0}"
            if ID not in phase_vigilant['simulation_IDs']:
                # simulate image
                image=simulator.compute_CR_ring( CR_ring_angle=phi_CR, R0_pixels=R0, Z=0, w0_pixels=w0)
                phase_vigilant['simulation_IDs'].append(ID)
                phase_vigilant['GT_R0s'].append(R0)
                phase_vigilant['GT_w0s'].append(w0)
                
                # get iX image
                image = np.where( image<=(max_intensity*saturation), image, max_intensity*saturation)
                image = compute_raw_to_centered_iX(image, X)
                
                # run gradient algorithm
                # charge the image
                image_container[0]=image.astype(np.float64)
                image_names[0]=ID
                # charge the image loader:
                image_loader.import_converted_images_as_array(image_container, image_names)
                
                optimal_masked_gravs={}
                optimal_radii={}
                optimal_angle={}
                grav=compute_intensity_gravity_center(image)
                # run both fibonacci and quadratic and then compute the average as the desired optimal
                gradient_algorithm.reInitialize(image_loader)
                gradient_algorithm.quadratic_fit_search(precision_quadratic, max_it_quadratic, cost_tolerance_quadratic)
                optimal_masked_gravs['quad'] = gradient_algorithm.masked_gravs[f"Quadratic_Search_{ID}"]
                optimal_radii['quad'] = gradient_algorithm.optimals[f"Quadratic_Search_{ID}"]
                optimal_angle['quad'] = gradient_algorithm.angles[f"Quadratic_Search_{ID}"]
                gradient_algorithm.reInitialize(image_loader)
                gradient_algorithm.fibonacci_ratio_search(precision_fibonacci, max_points_fibonacci, cost_tolerance_fibonacci)
                optimal_masked_gravs['fibo'] = gradient_algorithm.masked_gravs[f"Fibonacci_Search_{ID}"]
                optimal_radii['fibo'] = gradient_algorithm.optimals[f"Fibonacci_Search_{ID}"]
                optimal_angle['fibo'] = gradient_algorithm.angles[f"Fibonacci_Search_{ID}"]

                masked_grav=(optimal_masked_gravs['quad']+optimal_masked_gravs['fibo'])/2.0
                
                phase_vigilant['Ds'].append((optimal_radii['quad']+optimal_radii['fibo'])/2)
                phase_vigilant['grav'].append(grav.tolist())
                phase_vigilant['masked_grav'].append(masked_grav.tolist())
                # since we know that phiCR=0 we could input the exact profile of the main axis...but maybe for 
                # generality afterwards it will be nice to do it as there
                slope=(masked_grav[0]-grav[0])/(masked_grav[1]-grav[1])
                mask=(rows<( slope*(cols-masked_grav[1]) +masked_grav[0]+pix_spacing )) & (rows>( slope*(cols-masked_grav[1]) +masked_grav[0]-pix_spacing ))

                filtered_image=np.where(mask, image, 0)
                phase_vigilant['profiles'].append((np.sum(filtered_image,axis=0)).tolist())
                phase_vigilant['slope'].append(slope)
                
                # it is good to record the error performed in the angle as a measure of how accurate the gradient algorithm was detecting the main axis
                phase_vigilant['angle_error'].append(max(abs(optimal_angle['fibo']-phi_CR), abs(optimal_angle['quad']-phi_CR)))
                # we save the progess (in order to be able to quit and resume)
                json.dump(phase_vigilant, open( vig_path, "w"))
                print(f"{i}-th Simulated")
                i+=1

# We print the maximum error that happened in the angle detection, as a sanity check
print(f"Maximum error made in angle detection is {max(phase_vigilant['angle_error'])}")

# We pass to the next stage
if phase_vigilant['stage']==0:
    phase_vigilant['stage']=2
    json.dump(phase_vigilant, open( vig_path, "w"))
